In [1]:
import rootutils

rootutils.setup_root("./", indicator=".project-root", pythonpath=True)

import torch
import hydra
import time

import glob
import inspect
import yaml
import re
import os
from omegaconf import OmegaConf
from topobench.transforms import liftings
from topobench.data.preprocessor import PreProcessor

import memory_profiler
from memory_profiler import profile
%load_ext autoreload
%autoreload 2

In [2]:
hydra.initialize(config_path="../configs", job_name="job")

/tmp/ipykernel_12845/2472712977.py:1: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(config_path="../configs", job_name="job")


hydra.initialize()

In [3]:
cfg = hydra.compose(config_name="run.yaml", overrides=["model=hypergraph/unignn2", "dataset=hypergraph/coauthorship_cora"], return_hydra_config=True)

In [4]:
n_liftings = 0
classes_to_remove = ['Path', 'ModuleExportsManager', 'Any']
modules = [name for name, obj in inspect.getmembers(liftings) if inspect.ismodule(obj)]
classes_list = {}
for module in modules:
    if module.startswith("graph") or module.startswith("pointcloud"):
        domain2domain_module = getattr(liftings, module)
        domain2domain_classes = [name for name, obj in inspect.getmembers(domain2domain_module) if inspect.isclass(obj) and (name not in classes_to_remove and not module.lower() in name.lower())]
        print(f"Module: {module}")
        print(f"Classes: {domain2domain_classes}")
        classes_list[module] = domain2domain_classes
        n_liftings += len(domain2domain_classes)
print(f"Total number of liftings: {n_liftings}")

Module: graph2cell
Classes: ['CellCycleLifting', 'DiscreteConfigurationComplexLifting']
Module: graph2combinatorial
Classes: []
Module: graph2hypergraph
Classes: ['ExpanderGraphLifting', 'HypergraphFormanRicciCurvatureLifting', 'HypergraphKHopLifting', 'HypergraphKNNLifting', 'HypergraphKernelLifting', 'MapperLifting', 'ModularityMaximizationLifting']
Module: graph2simplicial
Classes: ['LatentCliqueLifting', 'NeighborhoodComplexLifting', 'SimplicialCliqueLifting', 'SimplicialDnDLifting', 'SimplicialEccentricityLifting', 'SimplicialGraphInducedLifting', 'SimplicialKHopLifting', 'SimplicialLineLifting', 'SimplicialVietorisRipsLifting']
Module: pointcloud2hypergraph
Classes: ['MoGMSTLifting', 'PointNetLifting', 'VoronoiLifting']
Module: pointcloud2simplicial
Classes: ['AlphaComplexLifting', 'RandomFlagComplexLifting']
Total number of liftings: 23


## Graph2\<domain\> Liftings

In [5]:
def best_match(module_name, config_folder):
    configs = glob.glob(f"{config_folder}/*.yaml")
    matches = [0 for _ in range(len(configs))]
    for i, config in enumerate(configs):
        name = os.path.basename(config)
        #remove the extension
        name = re.sub(r"\.yaml$", "", name)
        name_parts = name.split("_")
        for part in name_parts:
            if part in module_name.lower() and part != "graph":
                matches[i] += 1
    if not torch.any(torch.tensor(matches)>0):
        return None
    best_match = max(matches)
    if best_match == 0:
        print(f"No match found for {module_name} with {configs}")
        return None
    best_match_index = matches.index(best_match)
    return configs[best_match_index]

In [6]:
def print_transformed_data_info(data, transform_config):
    # Print all keys and values under transform_config["transform"]
    print("Transform Configuration:")
    for key, value in transform_config["transform"].items():
        print(f"  {key}: {value}")
    
    # Find the maximum rank by checking incidence_{i} attributes
    i = 0
    while hasattr(data, f"x_{i}"):
        incidence_matrix = getattr(data, f"x_{i}")
        print(f"Number of rank {i} cells: {incidence_matrix.shape[0]}")
        i += 1
    if hasattr(data, 'x_hyperedges'):
        print(f"Number of rank 1 cells: {data.x_hyperedges.shape[0]}")

In [7]:
@profile
def PreProcessorWrapper(dataset, dataset_dir, transform_config):
    t0 = time.time()
    processed_dataset = PreProcessor(dataset, dataset_dir, transform_config)
    time_elapsed = time.time() - t0
    print(f"Time elapsed: {time_elapsed:.2f} s.")
    data = processed_dataset.data
    print_transformed_data_info(data, transform_config)
    return None
    

In [10]:
root_folder = "./datasets/"
cfg = hydra.compose(config_name="run.yaml", overrides=["dataset=graph/cocitation_cora", f"dataset.loader.parameters.data_dir={root_folder}"], return_hydra_config=True)
loader = hydra.utils.instantiate(cfg.dataset.loader)

if os.path.exists(root_folder):
    os.system(f"rm -rf {root_folder}")
dataset, dataset_dir = loader.load()

# avoid_liftings = ['Discrete', 'Expander', 'Latent', 'Neighborhood', 'DnD', 'Eccentricity', 'GraphInduced', 'Line', 'VietorisRips'] #Discrete/Latent/GraphInduced are too slow, Expander could not converge (?) so it raised an error, Neighborhood/DnD/Line out of memory, Eccentricity doesn't work with disconnected graphs, VietorisRips crashes
avoid_liftings = []
# Apply transform
for module, classes in classes_list.items():
    if module.startswith("pointcloud"):# and module.endswith("simplicial"):
        for c in classes:
            config_folder = f"../configs/transforms/liftings/{module}"
            config_file = best_match(c, config_folder)
            avoid_this = False
            for avoid in avoid_liftings:
                if avoid in c:
                    avoid_this = True
                    break
            if avoid_this:
                continue
            print("#"*16, f"Class {c}", "#"*16)
            print(f"Using config file: {config_file}")
            transform_config = OmegaConf.load(config_file)
            transform_config = OmegaConf.to_container(transform_config, resolve=True)
            mem_usage = memory_profiler.memory_usage((PreProcessorWrapper, (dataset, dataset_dir, {"transform" :transform_config})), max_usage=True)
            print(f"Max memory usage: {mem_usage}")
            # PreProcessorWrapper(dataset, dataset_dir, {"transform" :transform_config})
            pass

################ Class MoGMSTLifting ################
Using config file: ../configs/transforms/liftings/pointcloud2hypergraph/mogmst.yaml


Processing...
Done!
Processing...


ERROR: Could not find file /tmp/ipykernel_12845/2743415116.py


: 

# MUTAG

################ Class CellCycleLifting ################
Using config file: ../configs/transforms/liftings/graph2cell/cycle.yaml
Downloading https://www.chrsmrrs.com/graphkerneldatasets/MUTAG.zip
Processing...
Done!
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 3.65 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: CellCycleLifting
  complex_dim: 3
  max_cell_length: 18
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  neighborhoods: None
Number of rank 0 cells: 3371
Number of rank 1 cells: 3721
Number of rank 2 cells: 538
/usr/local/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The data of the dataset is already cached, so any modifications to `data` will not be reflected when accessing its elements. Clearing the cache now by removing all elements in `dataset._data_list`. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
Max memory usage: 739.42578125
################ Class DiscreteConfigurationComplexLifting ################
Using config file: ../configs/transforms/liftings/graph2cell/discrete_configuration_complex.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 54.58 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: DiscreteConfigurationComplexLifting
  k: 2
  feature_aggregation: sum
  feature_lifting: ProjectionSum
  complex_dim: 3
  preserve_edge_attr: False
  neighborhoods: None
Number of rank 0 cells: 61010
Number of rank 1 cells: 128252
Number of rank 2 cells: 65146
Number of rank 3 cells: 0
Max memory usage: 759.42578125
################ Class ExpanderGraphLifting ################
Using config file: ../configs/transforms/liftings/graph2hypergraph/expander_graph.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
/usr/local/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The data of the dataset is already cached, so any modifications to `data` will not be reflected when accessing its elements. Clearing the cache now by removing all elements in `dataset._data_list`. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
Time elapsed: 1.74 s.
Transform Configuration:
  transform_type: lifting
  transform_name: ExpanderGraphLifting
  node_degree: 2
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  complex_dim: 1
  neighborhoods: None
Number of rank 0 cells: 3371
Max memory usage: 752.125
################ Class HypergraphFormanRicciCurvatureLifting ################
Using config file: ../configs/transforms/liftings/graph2hypergraph/forman_ricci_curvature.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 1.04 s.
Transform Configuration:
  transform_type: lifting
  transform_name: HypergraphFormanRicciCurvatureLifting
  network_type: weighted
  threshold_type: quantile
  threshold_direction: above
  threshold: 0.1
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  complex_dim: 1
  neighborhoods: None
Number of rank 0 cells: 3371
Max memory usage: 751.88671875
################ Class HypergraphKHopLifting ################
Using config file: ../configs/transforms/liftings/graph2hypergraph/khop.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 0.90 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: HypergraphKHopLifting
  k_value: 1
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  complex_dim: 1
  neighborhoods: None
Number of rank 0 cells: 3371
Max memory usage: 752.19921875
################ Class HypergraphKNNLifting ################
Using config file: ../configs/transforms/liftings/graph2hypergraph/knn.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 0.81 s.
Transform Configuration:
  transform_type: lifting
  transform_name: HypergraphKNNLifting
  k_value: 3
  loop: True
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  complex_dim: 1
  neighborhoods: None
Number of rank 0 cells: 3371
Max memory usage: 753.60546875
################ Class HypergraphKernelLifting ################
Using config file: ../configs/transforms/liftings/graph2hypergraph/kernel.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 0.76 s.
Transform Configuration:
  transform_type: lifting
  transform_name: HypergraphKernelLifting
  kernel_graph: heat
  kernel_feat: identity
  t: 1
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  complex_dim: 1
  neighborhoods: None
Number of rank 0 cells: 3371
Max memory usage: 756.57421875
################ Class MapperLifting ################
Using config file: ../configs/transforms/liftings/graph2hypergraph/mapper.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 1.48 s.
Transform Configuration:
  transform_type: lifting
  transform_name: MapperLifting
  filter_attr: laplacian
  resolution: 10
  gain: 0.3
  filter_func: None
  preserve_edge_attr: False
  complex_dim: 1
  neighborhoods: None
Number of rank 0 cells: 3371
Max memory usage: 758.13671875
################ Class ModularityMaximizationLifting ################
Using config file: ../configs/transforms/liftings/graph2hypergraph/modularity_maximization.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 0.76 s.
Transform Configuration:
  transform_type: lifting
  transform_name: ModularityMaximizationLifting
  num_communities: 2
  k_neighbors: 3
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  complex_dim: 1
  neighborhoods: None
Number of rank 0 cells: 3371
Max memory usage: 760.94921875
################ Class LatentCliqueLifting ################
Using config file: ../configs/transforms/liftings/graph2simplicial/latent_clique.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 58.03 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: LatentCliqueLifting
  edge_prob_mean: 0.95
  edge_prob_var: 0.0001
  signed: True
  preserve_edge_attr: False
  feature_lifting: ProjectionSum
  complex_dim: 3
  neighborhoods: None
Number of rank 0 cells: 3371
Number of rank 1 cells: 7777
Number of rank 2 cells: 7084
Max memory usage: 761.41796875
################ Class NeighborhoodComplexLifting ################
Using config file: ../configs/transforms/liftings/graph2simplicial/neighborhood_complex.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 6.31 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: NeighborhoodComplexLifting
  signed: True
  feature_lifting: ProjectionSum
  complex_dim: 3
  preserve_edge_attr: False
  neighborhoods: None
Number of rank 0 cells: 3371
Number of rank 1 cells: 9509
Number of rank 2 cells: 3695
Number of rank 3 cells: 858
Max memory usage: 774.07421875
################ Class SimplicialCliqueLifting ################
Using config file: ../configs/transforms/liftings/graph2simplicial/clique.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 4.94 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: SimplicialCliqueLifting
  complex_dim: 3
  preserve_edge_attr: False
  signed: False
  feature_lifting: ProjectionSum
  neighborhoods: None
Number of rank 0 cells: 3371
Number of rank 1 cells: 3721
Number of rank 2 cells: 0
Number of rank 3 cells: 0
Max memory usage: 759.828125
################ Class SimplicialDnDLifting ################
Using config file: ../configs/transforms/liftings/graph2simplicial/dnd.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 33.98 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: SimplicialDnDLifting
  signed: True
  feature_lifting: ProjectionSum
  complex_dim: 3
  preserve_edge_attr: False
  neighborhoods: None
Number of rank 0 cells: 3371
Number of rank 1 cells: 30505
Number of rank 2 cells: 185200
Number of rank 3 cells: 0
Max memory usage: 767.953125
################ Class SimplicialEccentricityLifting ################
Using config file: ../configs/transforms/liftings/graph2simplicial/eccentricity.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 30.56 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: SimplicialEccentricityLifting
  signed: True
  feature_lifting: ProjectionSum
  complex_dim: 3
  preserve_edge_attr: False
  neighborhoods: None
Number of rank 0 cells: 3371
Number of rank 1 cells: 30505
Number of rank 2 cells: 185200
Number of rank 3 cells: 0
Max memory usage: 778.33984375
################ Class SimplicialGraphInducedLifting ################
Using config file: ../configs/transforms/liftings/graph2simplicial/graph_induced.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 186.77 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: SimplicialGraphInducedLifting
  complex_dim: 3
  preserve_edge_attr: False
  signed: True
  feature_lifting: ProjectionSum
Number of rank 0 cells: 3371
Number of rank 1 cells: 14905
Number of rank 2 cells: 21458
Number of rank 3 cells: 8864
Max memory usage: 763.83203125
################ Class SimplicialKHopLifting ################
Using config file: ../configs/transforms/liftings/graph2simplicial/khop.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 6.30 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: SimplicialKHopLifting
  max_k_simplices: 5000
  feature_lifting: ProjectionSum
  complex_dim: 3
  preserve_edge_attr: False
  neighborhoods: None
Number of rank 0 cells: 3371
Number of rank 1 cells: 9149
Number of rank 2 cells: 6786
Number of rank 3 cells: 0
Max memory usage: 763.83203125
################ Class SimplicialLineLifting ################
Using config file: ../configs/transforms/liftings/graph2simplicial/line.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 4.46 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: SimplicialLineLifting
  feature_lifting: ProjectionSum
  complex_dim: 3
  preserve_edge_attr: False
  neighborhoods: None
Number of rank 0 cells: 3721
Number of rank 1 cells: 5428
Number of rank 2 cells: 1358
Max memory usage: 763.83203125
################ Class SimplicialVietorisRipsLifting ################
Using config file: ../configs/transforms/liftings/graph2simplicial/vietoris_rips.yaml
ERROR: Could not find file /tmp/ipykernel_91043/2743415116.py
Processing...
Done!
Time elapsed: 26.64 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: SimplicialVietorisRipsLifting
  complex_dim: 3
  signed: True
  distance_threshold: 2.0
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
Number of rank 0 cells: 3371
Number of rank 1 cells: 9149
Number of rank 2 cells: 8066
Number of rank 3 cells: 1699
Max memory usage: 763.83203125

## Pointcloud
################ Class MoGMSTLifting ################
Using config file: ../configs/transforms/liftings/pointcloud2hypergraph/mogmst.yaml
Downloading https://www.chrsmrrs.com/graphkerneldatasets/MUTAG.zip
Processing...
Done!
Processing...
Done!
Time elapsed: 5.77 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: MoGMSTLifting
  min_components: None
  max_components: None
  random_state: None
  preserve_edge_attr: False
  complex_dim: 1
Number of rank 0 cells: 3371
Number of rank 1 cells: 1200
Max memory usage: 905.26953125
################ Class PointNetLifting ################
Using config file: ../configs/transforms/liftings/pointcloud2hypergraph/pointnet.yaml
/usr/local/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. The data of the dataset is already cached, so any modifications to `data` will not be reflected when accessing its elements. Clearing the cache now by removing all elements in `dataset._data_list`. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
ERROR: Could not find file /tmp/ipykernel_12845/2743415116.py
Processing...
Time elapsed: 0.37 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: PointNetLifting
  sampling_ratio: 0.25
  cluster_radius: 10
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  complex_dim: 1
Number of rank 0 cells: 3371
Number of rank 1 cells: 910
Max memory usage: 906.20703125
################ Class VoronoiLifting ################
Using config file: ../configs/transforms/liftings/pointcloud2hypergraph/voronoi.yaml
Done!
ERROR: Could not find file /tmp/ipykernel_12845/2743415116.py
Processing...
Done!
Time elapsed: 0.75 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: VoronoiLifting
  support_ratio: 0.005
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  complex_dim: 1
Number of rank 0 cells: 3371
Number of rank 1 cells: 188
Max memory usage: 906.36328125
################ Class AlphaComplexLifting ################
Using config file: ../configs/transforms/liftings/pointcloud2simplicial/alpha_complex_delaunay.yaml
ERROR: Could not find file /tmp/ipykernel_12845/2743415116.py
Processing...
Done!
Time elapsed: 2.83 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: AlphaComplexLifting
  complex_dim: 3
  alpha: -1
  preserve_edge_attr: False
  signed: True
  feature_lifting: ProjectionSum
Number of rank 0 cells: 3371
Number of rank 1 cells: 631
Number of rank 2 cells: 257
Number of rank 3 cells: 25
Max memory usage: 896.48828125
################ Class RandomFlagComplexLifting ################
Using config file: ../configs/transforms/liftings/pointcloud2simplicial/random_flag_complex.yaml
ERROR: Could not find file /tmp/ipykernel_12845/2743415116.py
Processing...
Done!
Time elapsed: 3.16 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: RandomFlagComplexLifting
  complex_dim: 3
  alpha: 1.5
  steps: 10
  signed: True
  feature_lifting: ProjectionSum
Number of rank 0 cells: 3371
Number of rank 1 cells: 4069
Number of rank 2 cells: 427
Number of rank 3 cells: 8
Max memory usage: 895.953125

# CORA

################ Class CellCycleLifting ################
Using config file: ../configs/transforms/liftings/graph2cell/cycle.yaml
ERROR: Could not find file /tmp/ipykernel_617/2743415116.py
Processing...
Time elapsed: 0.46 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: CellCycleLifting
  complex_dim: 3
  max_cell_length: 18
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  neighborhoods: None
Number of rank 0 cells: 2708
Number of rank 1 cells: 5278
Number of rank 2 cells: 1774
Max memory usage: 5054.5546875
################ Class HypergraphFormanRicciCurvatureLifting ################
Using config file: ../configs/transforms/liftings/graph2hypergraph/forman_ricci_curvature.yaml
Done!
/usr/local/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
Processing...
ERROR: Could not find file /tmp/ipykernel_617/2743415116.py
Time elapsed: 0.74 s.
Transform Configuration:
  transform_type: lifting
  transform_name: HypergraphFormanRicciCurvatureLifting
  network_type: weighted
  threshold_type: quantile
  threshold_direction: above
  threshold: 0.1
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  complex_dim: 1
  neighborhoods: None
Number of rank 0 cells: 2708
Number of rank 1 cells: 2220
Max memory usage: 4936.90234375
################ Class HypergraphKHopLifting ################
Using config file: ../configs/transforms/liftings/graph2hypergraph/khop.yaml
Done!
Processing...
ERROR: Could not find file /tmp/ipykernel_617/2743415116.py
Time elapsed: 0.59 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: HypergraphKHopLifting
  k_value: 1
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  complex_dim: 1
  neighborhoods: None
Number of rank 0 cells: 2708
Number of rank 1 cells: 2708
Max memory usage: 4945.55859375
################ Class HypergraphKNNLifting ################
Using config file: ../configs/transforms/liftings/graph2hypergraph/knn.yaml
Done!
Processing...
ERROR: Could not find file /tmp/ipykernel_617/2743415116.py
Time elapsed: 5.80 s.
Transform Configuration:
  transform_type: lifting
  transform_name: HypergraphKNNLifting
  k_value: 3
  loop: True
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  complex_dim: 1
  neighborhoods: None
Number of rank 0 cells: 2708
Number of rank 1 cells: 2708
Max memory usage: 4946.65234375
################ Class HypergraphKernelLifting ################
Using config file: ../configs/transforms/liftings/graph2hypergraph/kernel.yaml
Done!
Processing...
ERROR: Could not find file /tmp/ipykernel_617/2743415116.py
Time elapsed: 5.83 s.
Transform Configuration:
  transform_type: lifting
  transform_name: HypergraphKernelLifting
  kernel_graph: heat
  kernel_feat: identity
  t: 1
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  complex_dim: 1
  neighborhoods: None
Number of rank 0 cells: 2708
Number of rank 1 cells: 2349
Max memory usage: 5393.76953125
################ Class MapperLifting ################
Using config file: ../configs/transforms/liftings/graph2hypergraph/mapper.yaml
Done!
ERROR: Could not find file /tmp/ipykernel_617/2743415116.py
Processing...
Time elapsed: 19.51 s.
Transform Configuration:
  transform_type: lifting
  transform_name: MapperLifting
  filter_attr: laplacian
  resolution: 10
  gain: 0.3
  filter_func: None
  preserve_edge_attr: False
  complex_dim: 1
  neighborhoods: None
Number of rank 0 cells: 2708
Number of rank 1 cells: 10681
Max memory usage: 5345.40234375
################ Class ModularityMaximizationLifting ################
Using config file: ../configs/transforms/liftings/graph2hypergraph/modularity_maximization.yaml
Done!
Processing...
ERROR: Could not find file /tmp/ipykernel_617/2743415116.py
Time elapsed: 18.19 s.
Transform Configuration:
  transform_type: lifting
  transform_name: ModularityMaximizationLifting
  num_communities: 2
  k_neighbors: 3
  feature_lifting: ProjectionSum
  preserve_edge_attr: False
  complex_dim: 1
  neighborhoods: None
Number of rank 0 cells: 2708
Number of rank 1 cells: 2708
Max memory usage: 5206.69921875
################ Class SimplicialCliqueLifting ################
Using config file: ../configs/transforms/liftings/graph2simplicial/clique.yaml
Processing...
Done!
Processing...
ERROR: Could not find file /tmp/ipykernel_12845/2743415116.py
Time elapsed: 1.23 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: SimplicialCliqueLifting
  complex_dim: 3
  preserve_edge_attr: False
  signed: False
  feature_lifting: ProjectionSum
  neighborhoods: None
Number of rank 0 cells: 2708
Number of rank 1 cells: 5278
Number of rank 2 cells: 1630
Number of rank 3 cells: 220
Max memory usage: 841.25390625
################ Class SimplicialKHopLifting ################
Using config file: ../configs/transforms/liftings/graph2simplicial/khop.yaml
Done!
/usr/local/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
Processing...
ERROR: Could not find file /tmp/ipykernel_12845/2743415116.py
Time elapsed: 7.34 s.
Transform Configuration:
  _target_: topobench.transforms.data_transform.DataTransform
  transform_type: lifting
  transform_name: SimplicialKHopLifting
  max_k_simplices: 5000
  feature_lifting: ProjectionSum
  complex_dim: 3
  preserve_edge_attr: False
  neighborhoods: None
Number of rank 0 cells: 2708
Number of rank 1 cells: 19462
Number of rank 2 cells: 5000
Number of rank 3 cells: 0
Max memory usage: 1128.53125
Done!